### **에이전트를 활용한 RAG**

- 문서 검색을 통해 최신 정보에 접근, 검색결과를 답변으로 제공하는 Agent 
- 질문에 따라 문서를 검색하여 답변 Or 인터넷 검색 도구를 활용하여 답변하는 에이전트 실습
- Agent를 활용하여 RAG를 수행하는것이 Agentic RAG

In [ ]:
# Tavily Search 
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(k = 3)

# template = """ 

# 에스티이노베이션의 정보에 대해서 알려주세요   => 이렇게 템플릿 형식은 LLM이 있어야 가능

# 주소 : 
# 전화번호 :
# 홈페이지 :

# """

# 검색 
search.invoke('에스티이노베이션이라는 회사에 대해서 알려주세요')

### **일반적인 RAG**

In [18]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS


loader = PyMuPDFLoader("data/p_data.pdf")
docs = loader.load()

print(f'문서 페이지 수 : {len(docs)}')

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)

# text splitter1 문서 로드와 split를 한번에 수행 loader.load()를 생략 
# split_text = loader.load_and_split(text_splitter = text_splitter)

# text_splitter2 문서 로드와 split를 따로 수행
split_text = text_splitter.split_documents(docs)

print(f'분리된 텍스트 페이지 수 : {len(split_text)}')

# Embedding 모델 정의
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')

# VectorStore 정의
vector_store = FAISS.from_documents(split_text, embeddings)

# Retriever 정의(기본 Retriever)
retriever = vector_store.as_retriever(search_kwargs = {'k': 3})

retriever.invoke("삼성전자가 개발한 생성형 AI 관련 내용을 문서에서 찾아줘")

문서 페이지 수 : 23
분리된 텍스트 페이지 수 : 43


[Document(id='cfc4c74c-bfe6-4b1e-a6a6-ad7a351ed6e0', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': 'data/p_data.pdf', 'file_path': 'data/p_data.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 12}, page_content='SPRi AI Brief |  \n2023-12월호\n10\n삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개\nn 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스’를 공개\nn 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유\nKEY Contents\n£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\nn 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\n∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를

### **Tools Retreiver**

In [21]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever = retriever,
    name = 'pdf_search',
    description = 'PDF 문서에서 정보를 검색하는 도구'
)

### **Agent 도구 정의**

In [24]:
tools = [search, retriever_tool]

### **Agent 생성**

1. LLM 정의
2. Prompt 정의

chat_history : 멀티턴 대화 시 사용

In [25]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        # ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [27]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# Agent 정의
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 정의
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = False)  # verbose = False : 중간 단계 생략

### **Run Agent**

#### **VectorDB에 없는 웹 검색이 필요한 질문**

In [31]:
from langchain_teddynote.messages import AgentStreamParser

# 각 단계별 출력을 위한 파서 생성
agent_stream_parser = AgentStreamParser()

# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "2024년 프로야구 플레이오프 진출한 5개 팀을 검색하여 알려주세요."}
)

for step in result:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 2024년 프로야구 플레이오프 진출 팀
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`



[관찰 내용]
Observation: [{'title': '프로야구 포스트시즌 경기 일정 및 결과 총정리 2024 - 만개의 축제', 'url': 'https://filmk.tistory.com/entry/%ED%94%84%EB%A1%9C%EC%95%BC%EA%B5%AC-%ED%8F%AC%EC%8A%A4%ED%8A%B8%EC%8B%9C%EC%A6%8C-%EA%B2%BD%EA%B8%B0-%EC%9D%BC%EC%A0%95-%EB%B0%8F-%EA%B2%B0%EA%B3%BC-%EC%B4%9D%EC%A0%95%EB%A6%AC2024', 'content': '2024 한국 프로야구 플레이오프에서 승리한 팀은 KIA와 맞붙게 되며, 그 승부는 한국시리즈 진출을 결정짓는 중요한 대결이 될 것입니다. 각 팀이 전력을', 'score': 0.80698997}, {'title': '2024 KBO 한국 프로야구 포스트시즌 일정 순위 예매 (와일드카드 준 ...', 'url': 'https://m.blog.naver.com/qhadldhaus98/223605904423', 'content': '야구 골수팬들은 당연히\n팀 순위가 머리 속에 있을 수 있지만\n라이트 팬들은 상위권이랑\n내가 응원하는 팀만 알지 어느팀이 몇위였더라?\n잘 모르실 수 있습니다.\n일단 2024 프로야구 순위를 먼저 적어보겠습니다.\n\u200b\n1.KIA 타이거즈\n2.삼성 라이온즈\n3.LG 트윈스\n4.두산 베어스\n5.KT 위즈\n6.SSG 랜더스\n7.롯데 자이언츠\n8.한화 이글스\n9.NC 다이노스\n10.키움 히어로즈\n\u200b\n올 해 프로야구 포스트시즌에 진출한 팀들은\nKIA 타이거즈 삼성 라이온즈

#### **VectorDB에 있는 Retriever 가 필요한 질문**

In [33]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "삼성전자가 자체 개발한 생성형 AI 관련된 정보를 문서에서 찾아주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 삼성전자 생성형 AI
Log: 
Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`



[관찰 내용]
Observation: SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 

In [34]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "Attenion is all you need 라는 논문에 나온 Transformer 구조에 대해서 알려줘"}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: Attention is All You Need Transformer architecture
Log: 
Invoking: `tavily_search_results_json` with `{'query': 'Attention is All You Need Transformer architecture'}`



[관찰 내용]
Observation: [{'title': 'Attention Is All You Need - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Attention_Is_All_You_Need', 'content': '"Attention Is All You Need"[1] is a 2017 landmark[2][3] research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer, based on the attention mechanism proposed in 2014 by Bahdanau et al.[4] It is considered a foundational[5] paper in modern artificial intelligence, as the transformer approach has become the main architecture of large language models like those based on GPT.[6][7] At the time, [...] The paper is most well known for the introduction of the Transformer architecture, which forms the underlying architecture for m